In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np


In [ ]:
#!pip install missingno

# Extracción

## 1. Corroborar que las carpetas existan y, de lo contrario, crearlas


In [ ]:
%run scripts/verificar_crear_carpetas.py

## 2. Descargar los archivos si no existen

In [ ]:
%run scripts/descargar_archivos.py

## 3. Descomprimir los archivos

In [ ]:
%run scripts/descomprimir_archivos.py


## 4. Descomprimir los archivos sin errores

In [ ]:
%run scripts/procesar_archivos_gzip.py

## 5. Cargar los archivos en DataFrames y mostrar en formato Markdown


In [ ]:
# Especificamos la ruta de los archivos JSON con el prefijo "se"
carpeta = "datasets/original"
archivo1 = os.path.join(carpeta, "se_user_reviews.json")
archivo2 = os.path.join(carpeta, "se_users_items.json")
archivo3 = os.path.join(carpeta, "se_steam_games.json")

# Cargamos los archivos JSON en DataFrames
df_user_reviews = pd.read_json(archivo1, orient='records', lines=True)
df_users_items = pd.read_json(archivo2, orient='records', lines=True)
df_steam_games = pd.read_json(archivo3, orient='records', lines=True)

# Imprimimos los DataFrames en formato Markdown
print("### DataFrame: user_reviews")
print(df_user_reviews.head().to_markdown(index=False))
print("\n### DataFrame: users_items")
print(df_users_items.head().to_markdown(index=False))
print("\n### DataFrame: steam_games")
print(df_steam_games.head().to_markdown(index=False))


# TRANSFORMACIÓN

## 1. EDA steam_games.json

### Limpieza y Preprocesamiento de los Datos

1. Identificar valores nulos:

In [ ]:
# Verificar los valores nulos por columna
print(df_steam_games.isnull().sum())

In [ ]:
# Visualizar los datos faltantes
msno.matrix(df_steam_games)
plt.show()


**df_steam_games.isnull().sum():** Cuenta la cantidad de valores nulos en cada columna.

**msno.matrix():** Crea una matriz visual para mostrar los valores faltantes.

2. Tratar valores nulos:

Eliminar filas: Si hay pocas filas con muchos valores nulos, puedes eliminarlas:

In [ ]:
df_steam_games.dropna(inplace=True)

3. Verificar y corregir tipos de datos:

In [ ]:
# Ver los tipos de datos actuales
print(df_steam_games.dtypes)

In [ ]:
# Convertir tipos de datos
df_steam_games['price'] = pd.to_numeric(df_steam_games['price'], errors='coerce')
df_steam_games['release_date'] = pd.to_datetime(df_steam_games['release_date'], errors='coerce')

4. Buscar valores atípicos y consistencia:

- Valores atípicos:

In [ ]:
# Configurar el tamaño de la figura
plt.figure(figsize=(20, 6))

# Boxplot para detectar outliers en el precio
sns.boxplot(x=df_steam_games['price'])

# Mostrar el gráfico
plt.show()

Por lo general los valores de los juegos no suelen sobrepasar los 60 dolares, por lo que 

- Valores inconsistentes:

In [ ]:
# Filtrar por precios negativos
df_steam_games[df_steam_games['price'] < 0]

In [ ]:
# Filtrar por fechas en el futuro
df_steam_games[df_steam_games['release_date'] > pd.Timestamp.now()]

5. Valores

**Celda 1: Verificar valores nulos**

In [ ]:
print(df_steam_games.isnull().sum())

**Celda 2: Imputar valores nulos en la columna 'price'**

In [ ]:
df_steam_games['price'].fillna(df_steam_games['price'].median(), inplace=True)

**Celda 3: Verificar nuevamente valores nulos**

In [ ]:
print(df_steam_games.isnull().sum())

**Celda 4: Encontrar valores únicos en la columna 'price'**

In [ ]:
unique_prices = df_steam_games['price'].unique().sum()
print("Valores únicos en 'price':", unique_prices)

**Celda 5: Contar filas duplicadas**

In [ ]:
duplicates = df_steam_games.duplicated().sum()
print(f"Número de filas duplicadas: {duplicates}")

**Celda 6: Contar la frecuencia de cada categoría en la columna 'genre'**

In [ ]:
genre_counts = df_steam_games['genre'].value_counts()
print("Distribución de géneros:")
print(genre_counts)

**Celda 7: Agrupar géneros poco frecuentes (opcional)**

In [ ]:
threshold = 2
infrequent_genres = genre_counts[genre_counts <= threshold].index.tolist()

if infrequent_genres:
    df_steam_games.loc[df_steam_games['genre'].isin(infrequent_genres), 'genre'] = 'Other'
    print("\nGéneros después de agrupar los poco frecuentes:")
    print(df_steam_games['genre'].value_counts())